<picture>
  <!--Imagem para o tema dark-->
  <source media="(prefers-color-scheme: dark)" srcset="https://github.com/CatarinaAguiar3/Projeto_Sistema_de_Recomendacao_MovieLens/blob/main/Imagens/Banners/Dark_Titulo_0_Amostragem.png?raw=true">
  
  <!--Imagem para o tema light-->
  <source media="(prefers-color-scheme: light)" srcset="https://github.com/CatarinaAguiar3/Projeto_Sistema_de_Recomendacao_MovieLens/blob/main/Imagens/Banners/Titulo_0_Amostragem_v2.png?raw=true">

  <!--Imagem padrão (quando os temas dark e light não forem identificados -->
  <img src="https://github.com/CatarinaAguiar3/Projeto_Sistema_de_Recomendacao_MovieLens/blob/main/Imagens/Banners/Titulo_0_Amostragem_v2.png?raw=true">
</picture>

A seguir, será feita uma amostragem de 3% dos usuários. O que dará aproximadamente 1 milhão de linhas na tabela "ratings".

# **Importar Bibliotecas**

In [1]:
import pandas as pd
import os
import numpy as np

## **Definir Diretório**

In [2]:
os.chdir("C:/0.Projetos/5.Sistema_de_Recomendação_MovieLens_2")

## **Carregar Arquivos**

In [5]:
movies = pd.read_csv("Datasets/0.Datasets_Originais/movies.csv")

In [11]:
ratings = pd.read_csv('Datasets/0.Datasets_Originais/ratings.csv')
#ratings = pd.read_csv('Data/Dataset_original/ratings.csv', sep=',', parse_dates=['timestamp'])

In [4]:
tags = pd.read_csv("Datasets/0.Datasets_Originais/tags.csv")
#tags = pd.read_csv("Datasets/0.Datasets_Originais/tags.csv",  sep=',', parse_dates=['timestamp'])

In [17]:
#NÃO SERÁ USADA
#links = pd.read_csv("Data/Dataset_original/links.csv")

In [6]:
genome_tags = pd.read_csv("Datasets/0.Datasets_Originais/genome-tags.csv")

In [7]:
genome_scores = pd.read_csv("Datasets/0.Datasets_Originais/genome-scores.csv")

In [17]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [18]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


In [46]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [21]:
tags.head()

,userId,movieId,tag,timestamp
0,10,260,good vs evil,1430666558
1,10,260,Harrison Ford,1430666505
2,10,260,sci-fi,1430666538
3,14,1221,Al Pacino,1311600756
4,14,1221,mafia,1311600746


In [8]:
genome_tags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [23]:
genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.03200
1,1,2,0.02225
2,1,3,0.07000
3,1,4,0.05900
4,1,5,0.12300


# **Amostragem da tabela ratings**
Faremos uma amostragem de 3% dos usuários da tabela `ratings` e em seguida, vamos utilizá-la como referência na amostragem das outras tabelas. 

In [16]:
# Amostragem com 3% dos IDs dos usuários
amostragem1 = np.random.choice(ratings['userId'].unique(),
                                size=int(len(ratings['userId'].unique()) * 0.03),
                                replace=False)


In [17]:
# Filtrar a tabela 'ratings' com os IDs selecionados
amostragem_ratings = ratings.loc[ratings['userId'].isin(amostragem1)]

In [18]:
# Filtrar a tabela 'ratings' com os IDs que NÃO foram selecionados
amostragem_ratings_complementar = ratings.loc[~ratings['userId'].isin(amostragem1)]

In [19]:
# Resumo da tabela com os IDs selecionados
print('Existem {} linhas , {} usuários e {} filmes'
    .format(len(amostragem_ratings),amostragem_ratings['userId'].nunique(),amostragem_ratings['movieId'].nunique()))

Existem 1028502 linhas , 9929 usuários e 26770 filmes


In [20]:
# Resumo da  tabela  com os IDs que NÃO foram selecionados
print('Existem {} linhas , {} usuários e {} filmes'
    .format(len(amostragem_ratings_complementar),amostragem_ratings_complementar['userId'].nunique(),amostragem_ratings_complementar['movieId'].nunique()))

Existem 32803660 linhas , 321046 usuários e 82777 filmes


## **Salvar a tabela**

In [23]:
amostragem_ratings_complementar.to_pickle("Datasets/1.Datasets_Amostragem/amostragem_ratings_complementar.pickle", compression="gzip")

In [22]:
amostragem_ratings.to_pickle("Datasets/1.Datasets_Amostragem/amostragem_ratings.pickle", compression="gzip")

In [8]:
#amostragem_ratings= pd.read_pickle("Datasets/1.Datasets_Amostragem/amostragem_ratings.pickle", compression="gzip")

In [24]:
del ratings, amostragem_ratings_complementar

# **Amostragem da tabela movies**
Para a amostragem da tabela `movies`, faremos um filtro nos `userId` da tabela rating. 

In [17]:
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [25]:
# Selecionar os "movieId" da tabela "amostragem_ratings"
movieId = np.array(amostragem_ratings['movieId'])

In [26]:
# Filtrar os mesmo de IDs de "amostragem_ratings" na tabela "movies"
movies_amostragem = movies.loc[movies['movieId'].isin(movieId)]

In [27]:
movies_amostragem

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
86397,288557,Initial D: Third Stage (2001),Action|Animation|Romance
86427,288647,Everybody's Oma (2022),Documentary
86435,288669,Insidious: The Red Door (2023),Horror|Mystery|Thriller
86439,288679,The Out-Laws (2023),Action|Comedy|Romance


In [29]:
# Salvar a tabela "movies_amostragem"
movies_amostragem.to_pickle("Datasets/1.Datasets_Amostragem/movies_amostragem.pickle", compression="gzip")

In [30]:
del movies

# **Amostragem da tabela tags**

In [47]:
tags.head()

,userId,movieId,tag,timestamp
0,10,260,good vs evil,1430666558
1,10,260,Harrison Ford,1430666505
2,10,260,sci-fi,1430666538
3,14,1221,Al Pacino,1311600756
4,14,1221,mafia,1311600746


In [49]:
amostragem_ratings.head()

,userId,movieId,rating,timestamp
213,5,47,5.0,1029389303
214,5,175,4.0,1029389417
215,5,257,4.0,1029389115
216,5,318,4.0,1029389280
217,5,319,4.0,1029389327


In [43]:
# Selecionar as colunas de interesse
amostragem_ratings1 = amostragem_ratings[['userId', 'movieId']]

In [44]:
# Filtrar a tabela "tags"
tags_amostragem = amostragem_ratings1.merge(tags, on=["movieId", "userId"], how="inner")

In [45]:
# Olhar a tabela criada
tags_amostragem 

,userId,movieId,tag,timestamp
0,134,89898,advertising,1521614243
1,134,89898,drugs,1521614238
2,134,89898,Russia,1521614240
3,134,182899,Action,1514209025
4,134,182899,Asia,1514209050
...,...,...,...,...
51294,330008,79091,nerds,1433358037
51295,330008,79091,supervillain,1433358009
51296,330008,81564,animation,1433357978
51297,330008,81564,funny,1433357989


In [50]:
# Salvar a tabela
tags_amostragem.to_pickle("Datasets/1.Datasets_Amostragem/tags_amostragem.pickle", compression="gzip") 

# **Amostragem da tabela genome_tags**

In [51]:
genome_tags.head(3)

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century


In [54]:
# Selecionar as "tagId" que serão usadas no filtro
tag_array = np.array(tags_amostragem["tag"])

In [ ]:
# Filtrar os "tag" da amostragem
genome_tags_amostragem = genome_tags.loc[genome_tags["tag"].isin(tag_array)]

In [56]:
# Olhar a tabela criada
genome_tags_amostragem

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s
...,...,...
1123,1124,writing
1124,1125,wuxia
1125,1126,wwii
1126,1127,zombie


In [57]:
# Salvar a tabela "genome_tags_amostragem"
genome_tags_amostragem.to_pickle("Datasets/1.Datasets_Amostragem/genome_tags_amostragem.pickle", compression="gzip")

In [58]:
del genome_tags

# **Amostragem da tabela genome_scores**

In [9]:
genome_tags_amostragem = pd.read_pickle("Datasets/1.Datasets_Amostragem/genome_tags_amostragem.pickle", compression="gzip")

In [29]:
genome_scores.head(3)

,movieId,tagId,relevance
0,1,1,0.03200
1,1,2,0.02225
2,1,3,0.07000


In [10]:
# Selecionar as "tagId" que serão usadas no filtro
tagId_array = np.array(genome_tags_amostragem["tagId"])

In [11]:
# Filtrar os "tagId" da amostragem
genome_scores_amostragem = genome_scores.loc[genome_scores["tagId"].isin(tagId_array)]

In [12]:
# Olhar a tabela criada
genome_scores_amostragem

,movieId,tagId,relevance
0,1,1,0.03200
1,1,2,0.02225
2,1,3,0.07000
3,1,4,0.05900
4,1,5,0.12300
...,...,...,...
18472123,288167,1124,0.09875
18472124,288167,1125,0.02950
18472125,288167,1126,0.02275
18472126,288167,1127,0.11225


In [14]:
genome_scores_amostragem.to_pickle("Datasets/1.Datasets_Amostragem/genome_scores_amostragem.pickle", compression="gzip")

# **⚠ No próximo Notebook será utilizado os arquivos:**
>  - amostragem_rating
>  - movies_amostragem
>  - tags_amostragem
>  - genome_tags_amostragem
>  - genome_scores_amostragem